# Übung 3

## 2) Textklassifikation mit multinomialen Bayesklassifikator

## 2a) Herunterladen und Entpacken der Datenbank

In [43]:
import pandas as pd
import numpy as np
import matplotlib as mp
import tarfile
import os.path
import urllib.request
import skimage
import scipy as sp
from random import shuffle

In [44]:
filename = "20news-18828.tar.gz"
url = "http://qwone.com/~jason/20Newsgroups/" + filename
if not os.path.isfile("./" + filename):
    urllib.request.urlretrieve(url, filename)
    dateDownloaded = !date
    print(dateDownloaded)

In [45]:
tar = tarfile.open("./" + filename)
tar.extractall()
tar.close()
foldername = filename.replace(".tar.gz", "")

## 2b) ...

In [46]:
import codecs
newsgroup_names = ["alt.atheism", "comp.graphics", "sci.space", "talk.religion.misc"]
newsgroups = {}
for newsgroup_name in newsgroup_names:
    newsgroup_folder = foldername+"/"+newsgroup_name
    text_names = os.listdir(newsgroup_folder)
    for text_name in text_names:
        with open(newsgroup_folder+"/"+text_name, "r",encoding='ISO-8859-1') as f:
            text = f.read()
            newsgroups.update({text_name : [newsgroup_name, text]})
newsgroups
len(newsgroups)

3387

In [47]:
for newsgroup_key in newsgroups:
    not_used, not_used, stripped_text = newsgroups[newsgroup_key][1].partition('\n\n')
    newsgroups.update({newsgroup_key : [newsgroups[newsgroup_key][0],stripped_text]})
newsgroups
len(newsgroups)

3387

## 2c)

In [48]:
import re
word_count_newsgroups = {}
for newsgroup_key in newsgroups:
    splitted_text = re.compile(r"(?u)\b\w\w+\b").findall(newsgroups[newsgroup_key][1].lower())
    word_count = {i:splitted_text.count(i) for i in splitted_text}
    word_count_newsgroups.update({newsgroup_key : word_count})

In [50]:
word_count_all = {}
for word_count_newsgroups_key in word_count_newsgroups:
    for single_word_count_key in newsgroups_splitted[word_count_newsgroups_key]:
        #print(single_word_count_key, newsgroups_splitted[word_count_newsgroups_key][single_word_count_key] )
        if single_word_count_key not in word_count_all:
            word_count_all.update({single_word_count_key : newsgroups_splitted[word_count_newsgroups_key][single_word_count_key]})
        else:
            multiple_word_count = newsgroups_splitted[word_count_newsgroups_key][single_word_count_key] + word_count_all[single_word_count_key]
            word_count_all.update({single_word_count_key : multiple_word_count})
len(word_count_all)

41777

## 2d)

In [59]:
def insert_words(word_count_classes, class_key, word_count):
    if class_key not in word_count_classes:
        word_count_classes.update({class_key : word_count})
    else:
        for word in word_count:
            if word not in word_count_classes[class_key]:
                word_count_classes[class_key].update({word : word_count[word]})
            else:
                mulitple_word_count = word_count_classes[class_key][word] + word_count[word]
                word_count_classes[class_key].update({word : multiple_word_count})
    return word_count_classes

In [71]:
from random import shuffle
newsgroups_keys_shuffled = list(newsgroups.keys())
shuffle(newsgroups_keys_shuffled)
split_size = int(len(newsgroups_keys_shuffled)*0.6)
word_count_classes_training = {}
word_count_classes_test = {}
word_count_text_training = {}
word_count_text_test = {}
for newsgroup_key, index in zip(newsgroups_keys_shuffled,range(0,len(newsgroups_keys_shuffled))):
    splitted_text = re.compile(r"(?u)\b\w\w+\b").findall(newsgroups[newsgroup_key][1].lower())
    word_count = {i:splitted_text.count(i) for i in splitted_text}
    class_key = newsgroups[newsgroup_key][0]
    if index < split_size:                            
        word_count_classes_training=insert_words(word_count_classes_training, class_key, word_count)
        word_count_text_training = insert_words(word_count_text_training, newsgroup_key, word_count)
    else:
        word_count_classes_test=insert_words(word_count_classes_test, class_key, word_count)
        word_count_text_test = insert_words(word_count_text_test, newsgroup_key, word_count)
                
for class_key in word_count_classes_training:
    print(class_key, len(word_count_classes_training[class_key]))
for class_key in word_count_classes_test:
    print(class_key, len(word_count_classes_test[class_key]))    

sci.space 15350
talk.religion.misc 11925
alt.atheism 11784
comp.graphics 13896
sci.space 12129
talk.religion.misc 9269
alt.atheism 10028
comp.graphics 11295


In [57]:
#word_count_newsgroups words per text {newsgroup:{word:count}}
#word_count_all all words over all texts {word:count}


#word_count_classes_test words per class {class:{word:count}}
#word_count_classes_training words per class {class:{word:count}}
#word_count_text_test words per text {newsgroup:{words:count}}
#word_count_text_training words per text {newsgroup:{words:count}}

## A-Priori Wahrscheinlichkeit dass es sich um eine bestimmte Klasse handelt

In [73]:
apriori_classes = {}
total_words_in_class = 0

for class_key in word_count_classes_training:
    total_words_in_class += len(word_count_classes_training[class_key])

for class_key in word_count_classes_training:
    apriori_classes[class_key] = len(word_count_classes_training[class_key]) / total_words_in_class
    
apriori_classes

{'alt.atheism': 0.22252856198659238,
 'comp.graphics': 0.26241148144651116,
 'sci.space': 0.28986875649136057,
 'talk.religion.misc': 0.22519120007553584}

In [77]:
trainings_matrix = {}
for class_key, word_dict in word_count_classes_training.items():
    number_words_in_class = len(word_dict)
    trainings_matrix[class_key] = {}
    for word in word_count_all:
        word_frequency = 0
        if word in word_dict:
            word_frequency = word_dict[word]
        probability = (word_frequency + 1) / (len(word_count_classes_training[class_key]) + len(word_count_all))
        trainings_matrix[class_key][word] = probability
        
#trainings_matrix

## Konfiguration

In [ ]:
%load_ext version_information
%version_information numpy, pandas, matplotlib, skimage